# Background on likelihood inference

## Preamble

This document will serve as the mathematical background to Estimagic's likelihood inference functionality. 

## Introduction

Maximum-likelihood estimation seeks to maximize the likelihood function, defined as a function of an unknown parameter vector, **$\beta$** , conditional on the data provided. For simplicity, we represent such a function as: $$L(\beta|y_j, x_j)$$ where $y_j$ is the observed outcome variable for observation $j$ from the sample data and $x_j$ is the $j$th row of covariates for the same observation. Specifically, the function is called the **joint density** since the probability density function for each observation is seen as independent of one another and the distribution from which the observed outcome variable is generated stays the same for each observation [2]; thus, we seek to maximize the product of the densities $$\prod_{j=1}^{n} f(y_j| x_j, \beta) = L(\beta|y_j, x_j) $$ In practice, the log of the likelihood function is used because it is simpler to maximize. The logarithm is a monotonic transformation, thus the parameters that maximize the product of the densities will also maximize the sum of the log-likelihoods. $$\ln L(\beta|y_j, x_j) = \sum^n_{j=1} \ln f(y_j|x_j, \beta) $$ 

The functional form of $f$ depends on the distribution of the disturbance or outcome variable from the estimation model. For the present documentation, we consider the logit model and thus, a logistical distribution for our outcome variable. A logistical distribution is used to estimate outcomes where the probability of observing an outcome lies between 0 and 1 and the sum of the probabilities is equal to 1. Its log-functional form or log-cumulative density function is shown and replaces $f$ from the above expression: $$\ln \left(\frac{1}{1+e^{-(2y_j-1)(x_j \beta)}}\right)\longrightarrow \sum^n_{j=1} \ln \left(\frac{1}{1+e^{-(2y_j-1)(x_j \beta)}}\right)$$ 

In [10]:
def estimate_likelihood_obs(params, y, x, criterion, design_options):
    """Pseudo-log-likelihood contribution per individual.

    Args:
        params (pd.DataFrame): The index consists of the parmater names,
            the "value" column are the parameter values.
        y (np.array): 1d numpy array with the dependent variable
        x (np.array): 2d numpy array with the independent variables
        criterion (string): "logit" or "probit"
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)
            
    Returns:
        loglike (np.array): 1d numpy array with likelihood contribution per individual

    """
    q = 2 * y - 1
    if criterion=="logit":
        c = np.log(1 / (1 + np.exp(-(q * np.dot(x, params["value"])))))
        if "weight" in design_options.columns:
            return c * design_options["weight"].to_numpy()
        else:
            return c
    elif criterion=="probit":
        c = np.log(stats.norm._cdf(np.dot(q[:, None] * x, params["value"])))
        if "weight" in design_options.columns:
            return c * design_options["weight"].to_numpy()
        else:
            return c
    else:
        print("Criterion function is misspecified or not supported.")

def estimate_likelihood(params, y, x, criterion, design_options):
    """Pseudo-log-likelihood.

    Args:
        params (pd.DataFrame): The index consists of the parameter names,
            the "value" column are the parameter values.
        y (np.array): 1d numpy array with the dependent variable
        x (np.array): 2d numpy array with the independent variables
        criterion (string): "logit" or "probit"
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)
            
    Returns:
        loglike (np.array): 1d numpy array with sum of likelihood contributions

    """
    return estimate_likelihood_obs(params, y, x, criterion, design_options).sum()

To find the vector of parameters which maximize the log-likelihood function, we take first order conditions with respect to the parameter vector: $$G(\beta) = \sum^n_{j=1} \frac{\partial \ln L(\beta|y_j, x_j)}{\partial \beta} = 0$$ The partial derivative of the log-likelihood for all observations with respect to the vector of parameters makes the **score vector** and we express the sum over all scores as $G(\beta)$. The existence of a global maximum depends on the shape of the function we are estimating. Search for the maximum if the log-likelihood function is concave. Search for the minimum if the log-likelihood is convex (which occurs if you flip the sign of the function). 

To generate the parameters which maximize the log-likelihood function, we use Estimagic's `maximize` functionality. For maximum-likelihood estimations, there is no closed-form solution; thus, numerical optimization is requisite in estimating the set of parameters. To estimate, there are several methods. We use the L-BFGS-B algorithm. Running the algorithm returns a set of parameters which maximize the log-likelihood. 

In [13]:
def estimate_parameters(criterion, formulas, data, design_options, dashboard=False):
    """Estimate parameters that maximize log-likelihood function.

    Args:
        criterion (string): "logit" or "probit"
        formulas (str or list of strings): a list of strings to be used by 
            patsy to extract dataframes of the dep. and ind. variables
        data (pd.DataFrame): a pandas DataFrame
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)
        dashboard (bool): Switch on the dashboard

    Returns:
        res: parameter vector and other optimization results.

    """
    params, y, x = mle_processing(formulas, data)

    res = maximize(
        criterion=estimate_likelihood,
        params=params,
        algorithm='scipy_L-BFGS-B',
        criterion_kwargs={"y": y, 
                          "x": x, 
                          "criterion": criterion, 
                          "design_options": design_options
                          },
        dashboard=dashboard)
    return res

To extract the standard-errors, we first solve for the variance of the parameter vector. There are three methods to estimate the variance: 

(1) taking the first order Taylor-Series expansion of $G(\beta)$ and solving for $\hat{V}(\hat{\beta})$. The result is the following: $$\hat{V}(\hat{\beta}) = \left[I(\beta)\right]^{-1} = \left[-H^{-1}\hat{V}\{G(\hat{\beta})\}-H^{-1}\right]$$ for $\beta=\hat{\beta}$. $H$ is a $k+1 \times k+1$ matrix of second-derivatives of the log-likelihood $\left(\frac{\partial^2 \ln L(\beta|y_j, x_j)}{\partial \hat{\beta} \partial \hat{\beta}'}\right)$. We refer to this method as the sandwich estimator. 

(2) The observed information matrix, which is the inverse of the Hessian matrix. This returns the variance-covariance matrix of the paramters and square-rooting the diagonal of variances, returns the standard errors. 

(3) Outer product of gradients. Taking the inverse of the product of two jacobian matrices returns the variance-covariance matrix for the parameters and likewise, returns the standard errors from squarerooting the diagonal. $$\left[I(\beta)\right]^{-1} = \left[\sum^n_{j=1} g_j'g_j\right]^{-1} $$

In [5]:
def mle_jacobian(params, formulas, data, criterion, design_options):
    """Jacobian of the pseudo-log-likelihood function for each observation.

    Args:
        params (pd.DataFrame): The index consists of the parmater names,
            the "value" column are the parameter values.
        formulas (list): a list of strings to be used by patsy to extract
            dataframes of the dependent and independent variables
        data (pd.DataFrame): a pandas dataset
        criterion (string): "logit" or "probit"
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)

    Returns:
        jacobian (np.array): an n x k + 1-dimensional array of first
            derivatives of the pseudo-log-likelihood function w.r.t. the parameters
        
    """    
    y, x = dmatrices(formulas[0], data, return_type='dataframe')
    y = y[y.columns[0]] 
    
    if criterion=="logit":
        c = 1 / (1 + np.exp(-(np.dot(x, params["value"])))) 
        jacobian = (y - c)[:, None] * x 
        if "weight" in design_options.columns:
            weight = design_options["weight"].to_numpy()[:, None]
            weighted_jacobian = weight/weight.mean() * jacobian.to_numpy()
            return weighted_jacobian
        else:
            return jacobian.to_numpy()

    elif criterion=="probit":
        pdf = stats.norm._pdf(np.dot(x, params["value"])) 
        cdf = stats.norm._cdf(np.dot(x, params["value"]))
        jacobian = ((pdf / (cdf * (1 - cdf))) * (y - cdf))[:, None] * x
        if "weight" in design_options.columns:
            weight = design_options["weight"].to_numpy()[:, None]
            weighted_jacobian = weight/weight.mean() * jacobian.to_numpy()
            return weighted_jacobian
        else:
            return jacobian.to_numpy() 
    else:
        print("Criterion function is misspecified or not supported.")
       
    
def mle_hessian(params, formulas, data, criterion, design_options):
    """Hessian matrix of the pseudo-log-likelihood function.

    Args:
        params (pd.DataFrame): The index consists of the parmater names,
            the "value" column are the parameter values.
        formulas (list): a list of strings to be used by patsy to extract
            dataframes of the dependent and independent variables
        data (pd.DataFrame): a pandas dataset
        criterion (string): "logit" or "probit"
        design_options (pd.DataFrame): dataframe containing psu, stratum,
            population/design weight and/or a finite population corrector (fpc)
    Returns:
        hessian (np.array): a k + 1 x k + 1-dimensional array of second derivatives
            of the pseudo-log-likelihood function w.r.t. the parameters

    """    
    y, x = dmatrices(formulas[0], data, return_type='dataframe')
    y = y[y.columns[0]] 
    
    if criterion=="logit":
        if "weight" in design_options.columns:
            weight = design_options["weight"].to_numpy()
            c = 1 / (1 + np.exp(-(np.dot(x, params["value"])))) 
            return -np.dot(weight/weight.mean() * c * (1 - c) * x.T, x)
        else:
            c = 1 / (1 + np.exp(-(np.dot(x, params["value"])))) 
            return -np.dot(c * (1 - c) * x.T, x)
    elif criterion=="probit":
        q = 2 * y - 1
        pdf = stats.norm._pdf(np.dot(q[:, None] * x, params["value"]))
        cdf = stats.norm._cdf(np.dot(q[:, None] * x, params["value"]))
        delt = (pdf * q) / cdf
        mid = np.dot(x, params["value"]) + delt
        if "weight" in design_options.columns:
            weight = design_options["weight"].to_numpy()
            tranpose = (delt * mid)[:, None] * weight[:, None]/weight[:, None].mean() * x
            hessian = np.dot(tranpose.T, x)
        else:
            tranpose = (delt * mid)[:, None] * x
            hessian = np.dot(tranpose.T, x)
    
        return hessian
    else:
        print("Criterion function is misspecified or not supported.")
        
def observed_information_matrix(hessian):
    oim_var = np.linalg.inv(hessian)
    oim_se = np.sqrt(np.diag(oim_var))
    return oim_se, oim_var

def outer_product_of_gradients(jacobian):
    opg_var = np.linalg.inv(np.dot(jacobian.T, jacobian))
    opg_se = np.sqrt(np.diag(opg_var))
    return opg_se, opg_var

def sandwich_step(hessian, meat):
    """The sandwich estimator for variance estimation.
    
    Args:
        hessian (np.array): 2d numpy array with the hessian of the
            pseudo-log-likelihood function evaluated at `params`
        meat (np.array): the variance of the total scores
        
    Returns:
        se (np.array): a 1d array of k + 1 standard errors
        var (np.array): 2d variance-covariance matrix 
        
    """
    invhessian = np.linalg.inv(hessian)
    var = np.dot(np.dot(invhessian, meat), invhessian)
    se = np.sqrt(np.diag(var))
    return se, var

The inner component, $\hat{V}\{G(\hat{\beta})\}$, is the variance of a sum of partial derivatives. The variance of any sum is: $$\frac{n}{n-1} \sum^{n}_{j=1}(u_j-\overline{u_j})^2$$ The mean of the score vector is zero, thus $\overline{u_j} =0$. Thus, the robust variance estimator is: $$\hat{V}(\hat{\beta}) = \left[-H^{-1}\left(\frac{n}{n-1} \sum^{n}_{j=1}(u^{'}_ju_j)\right)-H^{-1}\right]$$

Given its formulation, this is known as a sandwich estimator. The use of this variance estimator is justified for independent observations. In the case where a group of observations are correlated, we have to adjust the estimator to take the sum over clusters compared to taking sums over individual observations. 

The cluster-robust variance is: $$\hat{V}(\hat{\beta}) = \left[-H^{-1}\left(\frac{n_c}{n_c-1} \sum^{n_c}_{j=1}\left(\sum_{j\in C_i}u_j\right)^{'}\left(\sum_{j\in C_i}u_j\right)\right)-H^{-1}\right]$$ where $n_c$ refers to the number of clusters and $\sum_{j\in C_i}u_j$ is the sum of the scores in cluster $j$. 